# Excel File Difference Analyzer

This notebook provides a robust function to compare two Excel files and generate a difference report.

In [ ]:
from pathlib import Path

import pandas as pd

In [ ]:
def diff_xlsx_files(file1_path, file2_path, output_path, join_columns=None):
    """
    Compare two Excel files and generate a difference report.
    
    Args:
        file1_path: Path to first Excel file (reference)
        file2_path: Path to second Excel file (comparison)
        output_path: Path for difference report Excel file
        join_columns: List of column names to use for joining (if None, uses row index)
        
    Returns:
        dict: Summary of differences found
        
    Raises:
        FileNotFoundError: If input files don't exist
        ValueError: If files have incompatible structures
    """
    # Convert to Path objects
    file1 = Path(file1_path)
    file2 = Path(file2_path)
    output = Path(output_path)
    
    # Validate input files exist
    if not file1.exists():
        raise FileNotFoundError(f"File not found: {file1}")
    if not file2.exists():
        raise FileNotFoundError(f"File not found: {file2}")
    
    # Create output directory if needed
    output.parent.mkdir(parents=True, exist_ok=True)
    
    try:
        # Read files
        df1 = pd.read_excel(file1)
        df2 = pd.read_excel(file2)
        
        print(f"File 1: {len(df1)} rows, {len(df1.columns)} columns")
        print(f"File 2: {len(df2)} rows, {len(df2.columns)} columns")
        
        # Check for empty dataframes
        if df1.empty and df2.empty:
            print("Warning: Both files are empty")
            summary = {"status": "both_empty", "differences": 0}
            return summary
        
        # Compare column structures
        cols1 = set(df1.columns)
        cols2 = set(df2.columns)
        common_cols = cols1.intersection(cols2)
        only_in_file1 = cols1 - cols2
        only_in_file2 = cols2 - cols1
        
        print(f"Common columns: {len(common_cols)}")
        if only_in_file1:
            print(f"Only in file 1: {list(only_in_file1)}")
        if only_in_file2:
            print(f"Only in file 2: {list(only_in_file2)}")
        
        # Prepare data for comparison
        if join_columns:
            # Validate join columns exist in both files
            missing_cols = set(join_columns) - common_cols
            if missing_cols:
                raise ValueError(f"Join columns not found in both files: {missing_cols}")
            
            # Merge on specified columns
            merged = pd.merge(df1, df2, on=join_columns, how='outer', 
                            suffixes=('_file1', '_file2'), indicator=True)
        else:
            # Compare by index if same shape
            if df1.shape == df2.shape and list(df1.columns) == list(df2.columns):
                merged = df1.copy()
                merged['_merge'] = 'both'
                for col in common_cols:
                    merged[f'{col}_file1'] = df1[col]
                    merged[f'{col}_file2'] = df2[col]
                    merged[f'{col}_diff'] = (df1[col] != df2[col])
            else:
                raise ValueError(
                    "Files have different shapes. Please specify join_columns for comparison."
                )
        
        # Create difference analysis
        differences = []
        
        if join_columns:
            # Analyze merge results
            only_file1 = merged[merged['_merge'] == 'left_only']
            only_file2 = merged[merged['_merge'] == 'right_only']
            both_files = merged[merged['_merge'] == 'both']
            
            # Create summary sheet
            summary_data = {
                'Metric': ['Rows only in file 1', 'Rows only in file 2', 'Common rows', 
                          'Total differences'],
                'Count': [len(only_file1), len(only_file2), len(both_files), 0]
            }
            
            # Check for value differences in common rows
            diff_count = 0
            for col in common_cols:
                if col not in join_columns:
                    col1 = f'{col}_file1'
                    col2 = f'{col}_file2'
                    if col1 in both_files.columns and col2 in both_files.columns:
                        diffs = both_files[col1] != both_files[col2]
                        diff_count += diffs.sum()
                        both_files[f'{col}_differs'] = diffs
            
            summary_data['Count'][3] = diff_count
            
        else:
            # Count cell-by-cell differences
            diff_count = 0
            for col in common_cols:
                diffs = merged[f'{col}_diff'].sum()
                diff_count += diffs
            
            summary_data = {
                'Metric': ['Total cell differences', 'Rows compared', 'Columns compared'],
                'Count': [diff_count, len(merged), len(common_cols)]
            }
        
        summary_df = pd.DataFrame(summary_data)
        
        # Write results to Excel with multiple sheets
        with pd.ExcelWriter(output, engine='openpyxl') as writer:
            # Summary sheet
            summary_df.to_excel(writer, sheet_name='Summary', index=False)
            
            if join_columns:
                # Detailed comparison
                if len(merged) > 0:
                    merged.to_excel(writer, sheet_name='Detailed_Comparison', index=False)
                
                # Rows only in file 1
                if len(only_file1) > 0:
                    only_file1.to_excel(writer, sheet_name='Only_in_File1', index=False)
                
                # Rows only in file 2
                if len(only_file2) > 0:
                    only_file2.to_excel(writer, sheet_name='Only_in_File2', index=False)
            else:
                # Cell-by-cell comparison
                merged.to_excel(writer, sheet_name='Cell_Comparison', index=False)
        
        # Prepare summary
        summary = {
            "status": "completed",
            "file1_rows": len(df1),
            "file2_rows": len(df2),
            "common_columns": len(common_cols),
            "differences": diff_count
        }
        
        if join_columns:
            summary.update({
                "only_in_file1": len(only_file1),
                "only_in_file2": len(only_file2),
                "common_rows": len(both_files)
            })
        
        print(f"Difference analysis completed. Found {diff_count} differences.")
        print(f"Report saved to: {output}")
        
        return summary
        
    except Exception as e:
        print(f"Error processing files: {e}")
        raise

## Usage Examples

### Example 1: Compare files by row index (same structure required)

In [ ]:
# Compare files with same structure by row index
file1_path = Path('data1.xlsx')
file2_path = Path('data2.xlsx')
output_path = Path('differences_by_index.xlsx')

try:
    summary = diff_xlsx_files(file1_path, file2_path, output_path)
    print("Comparison completed successfully!")
    print(f"Summary: {summary}")
except FileNotFoundError as e:
    print(f"File error: {e}")
except ValueError as e:
    print(f"Data error: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")

### Example 2: Compare files using specific join columns

In [ ]:
# Compare files using ID columns for matching
file1_path = Path('dataset1.xlsx')
file2_path = Path('dataset2.xlsx')
output_path = Path('differences_by_id.xlsx')
join_cols = ['id']  # Specify columns to match on

try:
    summary = diff_xlsx_files(file1_path, file2_path, output_path, join_columns=join_cols)
    print("Comparison completed successfully!")
    print(f"Summary: {summary}")
except FileNotFoundError as e:
    print(f"File error: {e}")
except ValueError as e:
    print(f"Data error: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")

## View Results

If comparison was successful, you can preview the summary:

In [ ]:
# Display summary if available
if 'summary' in locals():
    print("Difference Analysis Summary:")
    for key, value in summary.items():
        print(f"  {key}: {value}")
else:
    print("No comparison results available")

In [ ]:
# Optional: Load and preview the generated difference report
if 'output_path' in locals() and Path(output_path).exists():
    # Read summary sheet
    summary_df = pd.read_excel(output_path, sheet_name='Summary')
    print("Difference Report Summary:")
    display(summary_df)
else:
    print("No difference report file found")